In [1]:
def load_mnist(batch_size, path):

    if not os.path.exists(path): os.mkdir(path)
    trans = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,), (1.0,))])

    train_set = torchvision.datasets.MNIST(root=path, train=True, 
                                           transform=trans, download=True)
    test_set = torchvision.datasets.MNIST(root=path, train=False, 
                                          transform=trans, download=True)

    train_loader = torch.utils.data.DataLoader(
        dataset=train_set,
        batch_size=batch_size,
        shuffle=True)

    test_loader = torch.utils.data.DataLoader(
        dataset=test_set,
        batch_size=batch_size,
        shuffle = False)

    return train_loader, test_loader

In [50]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import torch


### Load dataset - Preprocessing
X, y = load_breast_cancer().data, load_breast_cancer().target
X = X.astype(np.float32)
y = y.reshape(-1, 1).astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2,
                                                   random_state=15)
# standardize data
Xs = np.std(X_train, axis=0, keepdims=True)
X_train /= Xs
X_test /= Xs


### Build network
IN_SIZE = 30
HIDDEN_SIZE = 200
OUT_SIZE = 1
LR=0.001

class Net(torch.nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(IN_SIZE , HIDDEN_SIZE)
        self.l2 = torch.nn.Linear(HIDDEN_SIZE, OUT_SIZE)

    def forward(self, x):
        x = torch.relu(self.l1(x))
        y_logits = self.l2(x)
        return y_logits

net = Net()
criterion = torch.nn.BCEWithLogitsLoss(reduction='sum')
opti = torch.optim.SGD(net.parameters(), lr=LR)

### Training
NEPOCHS = 50
BATCH_SIZE = 32

for epoch in range(NEPOCHS):

    p = np.random.permutation(len(X_train))
    X_train, y_train = X_train[p], y_train[p]
    for ki in range(0, len(X_train), BATCH_SIZE):
        X_batch = torch.from_numpy(X_train[ki:ki+BATCH_SIZE])
        y_batch = torch.from_numpy(y_train[ki:ki+BATCH_SIZE])
    
        net.zero_grad()
        y_logits = net(X_batch)
        loss = criterion(y_logits, y_batch)
        loss.backward()
        opti.step()
        
    X = torch.from_numpy(X_train)
    y = torch.from_numpy(y_train)
    y_logits = net(X)
    loss = criterion(y_logits, y)
    preds = torch.round(torch.sigmoid(y_logits))
    acc = y.eq(preds).sum().float() / len(X)
    print('Epoch {}: Loss = {}, Accuracy = {}'.format(epoch+1, 
                                                      loss.data,
                                                     acc))
    
    
### Evaluate
X = torch.from_numpy(X_test)
y = torch.from_numpy(y_test)
preds = torch.round(torch.sigmoid(net(X)))
acc = y.eq(preds).sum().float() / len(X)
print('Test Accuracy = {}'.format(acc))

Epoch 1: Loss = 158.02099609375, Accuracy = 0.901098906993866
Epoch 2: Loss = 118.58281707763672, Accuracy = 0.9098901152610779
Epoch 3: Loss = 100.8876953125, Accuracy = 0.9208791255950928
Epoch 4: Loss = 89.38525390625, Accuracy = 0.9252747297286987
Epoch 5: Loss = 79.79706573486328, Accuracy = 0.9384615421295166
Epoch 6: Loss = 73.24398803710938, Accuracy = 0.9450549483299255
Epoch 7: Loss = 70.56520080566406, Accuracy = 0.9296703338623047
Epoch 8: Loss = 70.55205535888672, Accuracy = 0.9516483545303345
Epoch 9: Loss = 61.68472671508789, Accuracy = 0.9494505524635315
Epoch 10: Loss = 73.28267669677734, Accuracy = 0.9494505524635315
Epoch 11: Loss = 54.28984069824219, Accuracy = 0.9626373648643494
Epoch 12: Loss = 52.314666748046875, Accuracy = 0.9626373648643494
Epoch 13: Loss = 52.99263000488281, Accuracy = 0.9516483545303345
Epoch 14: Loss = 48.72391891479492, Accuracy = 0.9648351669311523
Epoch 15: Loss = 50.39686965942383, Accuracy = 0.9494505524635315
Epoch 16: Loss = 44.828735